# Minimum imbalance problem

### Dependencies & utiliy methods

In [1]:
import numpy as np
import gurobipy as gb
from itertools import combinations, product
from math import floor
from functools import partial

# other solver engines
from ortools.graph import pywrapgraph
import networkx as nx

In [2]:
# samples from the control samples are indexed from 0 to n_prime
def extract_n_prime(L_prime, k):
    M = 0
    for i in range(k[0]):
        if len(L_prime[0][i]) > 0:
            M = max(M, np.max(L_prime[0][i]))
    return int(M) + 1


# number of treatment samples (and also cardinality of S)
def extract_n(l):
    return int(sum(l[0]))


def extract_k(l):
    return list(map(len, l))

### Brute force

In [3]:
def compute_imbalance(map_L_prime_to_value, S, l):
    s = 0
    for p in range(P):
        S = list(S)
        values, counts = np.unique(map_L_prime_to_value[p][S], return_counts=True)
        # we might be missing some values in values
        full_counts = np.zeros(len(l[p]), dtype=int)
        full_counts[values] = counts
        s += np.sum(np.abs(full_counts - l[p]))
    return s


def brute_force(l, L_prime):
    k = extract_k(l)
    n_prime = extract_n_prime(L_prime, k)
    n = extract_n(l)

    z = np.arange(n_prime)

    P = len(k)

    map_L_prime_to_value = []
    for p in range(P):
        ls = np.empty(n_prime, dtype=int)
        for i in range(k[p]):
            ls[L_prime[p][i]] = i
        map_L_prime_to_value.append(ls)

    m = 10000000
    S_m = None
    for S in combinations(z, n):
        mi = compute_imbalance(map_L_prime_to_value, S, l)
        if mi < m:
            m = mi
            S_m = [S]
        elif mi == m:
            S_m.append(S)
    return S_m, m

### Linear programming formulation

In [4]:
# each k[i] consecutive rows of A contain 1 in the j-th
# column if for the (i+1)-th covariate we have that
# z[j] belongs to the k-th bucket of the covariate i,
# where k is the offset from k[i]+sum(k[:i])
def compute_A(L_prime, k, n_prime):
    A = np.zeros((sum(k), n_prime), dtype=int)
    current_row = 0
    for p in range(P):
        Lp_prime = L_prime[p]
        for i in range(k[p]):
            A[current_row, Lp_prime[i]] = 1
            current_row += 1
    return A


def min_imbalance_solver(l, L_prime, verbose=False):
    min_imbalance = gb.Model()
    min_imbalance.modelSense = gb.GRB.MINIMIZE
    min_imbalance.setParam("outputFlag", 0)

    n = extract_n(l)
    k = extract_k(l)
    P = len(k)
    l = np.array(np.concatenate(l))
    n_prime = extract_n_prime(L_prime, k)

    # 1e
    z = min_imbalance.addMVar(n_prime, vtype=gb.GRB.BINARY)
    y = min_imbalance.addMVar(sum(k))

    A = compute_A(L_prime, k, n_prime)

    # 1b
    min_imbalance.addConstr(A @ z - l <= y)
    # 1c
    min_imbalance.addConstr(l - A @ z <= y)
    # 1d
    min_imbalance.addConstr(gb.quicksum(z) == n)

    # 1a
    min_imbalance.setObjective(gb.quicksum(y))

    min_imbalance.optimize()

    if verbose:
        if min_imbalance.status == 2:
            print("OK")
        else:
            print("Bad things happened")
    return z.x, sum(y.x)

### Alternative linear programming formulation

In [5]:
def min_imbalance_solver_alt(l, L_prime, verbose=False):
    min_imbalance = gb.Model()
    min_imbalance.modelSense = gb.GRB.MINIMIZE
    min_imbalance.setParam("outputFlag", 0)

    n = extract_n(l)
    k = extract_k(l)
    P = len(k)
    l = np.array(np.concatenate(l))
    n_prime = extract_n_prime(L_prime, k)

    assert P == 2

    # 2f
    z = min_imbalance.addMVar(n_prime, vtype=gb.GRB.BINARY)
    # 2e
    e = min_imbalance.addMVar(sum(k), lb=0.0)
    d = min_imbalance.addMVar(sum(k), lb=0.0)

    A = compute_A(L_prime, k, n_prime)

    for p in range(2):
        # smallest index for the covariate p
        bottom_index = sum(k[:p])
        # biggest index for the covariate p
        top_index = k[p] + bottom_index

        sl = slice(bottom_index, top_index)
        # 2b/2c
        min_imbalance.addConstr(A[sl] @ z + d[sl] - e[sl] == l[sl])

    # 2d
    min_imbalance.addConstr(gb.quicksum(e[: k[0]]) - gb.quicksum(d[: k[0]]) == 0)

    # 2a
    min_imbalance.setObjective(gb.quicksum(e) + gb.quicksum(d))

    min_imbalance.optimize()

    if verbose:
        if min_imbalance.status == 2:
            print("OK")
        else:
            print("Bad things happened")
    return z.x, sum(e.x) + sum(d.x)

### Minimum cost network flow (linear programming)

In [6]:
def compute_U(A, k):
    U = np.empty((k[0], k[1]), dtype=int)

    A1 = A[: k[0]] > 0
    A2 = A[k[0] :] > 0

    return np.count_nonzero(np.logical_and(A1[:, None], A2[None]), axis=-1)


def X_to_Z(A, k, X):
    U = np.empty((k[0], k[1]), dtype=int)

    A1 = A[: k[0]] > 0
    A2 = A[k[0] :] > 0

    B = np.logical_and(A1[:, None], A2[None])

    z = np.zeros(B.shape[2])
    for i1 in range(B.shape[0]):
        for i2 in range(B.shape[1]):
            # i1 \cap i2
            intersection = int(X[i1 * k[1] + i2])
            # we take the first intersection values in the intersection
            take = np.where(B[i1, i2])[0][:intersection]
            z[take] = 1
    return z


def min_imbalance_solver_mcnf(l, L_prime, verbose=False):
    min_imbalance = gb.Model()
    min_imbalance.modelSense = gb.GRB.MINIMIZE
    min_imbalance.setParam("outputFlag", 0)

    n = extract_n(l)
    k = extract_k(l)
    P = len(k)
    l = np.array(np.concatenate(l))
    n_prime = extract_n_prime(L_prime, k)

    assert P == 2

    A = compute_A(L_prime, k, n_prime)
    U = compute_U(A, k)

    # 3g (i2 changes faster than i1)
    x = min_imbalance.addMVar(U.shape[0] * U.shape[1], lb=0.0, ub=U.flatten(order="C"))
    # 3f
    e = min_imbalance.addMVar(sum(k), lb=0.0)
    d = min_imbalance.addMVar(sum(k), lb=0.0)

    for p in range(2):
        # 3b
        for i1 in range(k[0]):
            x_i1 = gb.quicksum(x[k[1] * i1 : k[1] * (i1 + 1)])
            min_imbalance.addConstr(x_i1 + d[i1] - e[i1] == l[i1])

        # 3c
        for i2 in range(k[1]):
            x_i2 = gb.quicksum(x[i2 :: k[1]])
            min_imbalance.addConstr(
                -x_i2 - d[k[0] + i2] + e[k[0] + i2] == -l[k[0] + i2]
            )

        # 3d
        min_imbalance.addConstr(gb.quicksum(e[: k[0]]) - gb.quicksum(d[: k[0]]) == 0)

        # 3e
        min_imbalance.addConstr(gb.quicksum(e[k[0] :]) - gb.quicksum(d[k[0] :]) == 0)

    # 3a
    min_imbalance.setObjective(gb.quicksum(e) + gb.quicksum(d))

    min_imbalance.optimize()

    if verbose:
        if min_imbalance.status == 2:
            print("OK")
        else:
            print("Bad things happened")
    return X_to_Z(A, k, x.x), sum(e.x) + sum(d.x)

### Minimum cost network flow (NetworkX)

In [7]:
infinity = 100000000

# we set all data explicitly because this will be used by another solver
def min_imbalance_network(l, L_prime):
    G = nx.DiGraph()

    G.add_node(0, demand=0)
    G.add_node(1, demand=0)

    n = extract_n(l)
    k = extract_k(l)
    P = len(k)
    assert P == 2

    for i in range(k[0]):
        G.add_node((0, i), demand=-l[0][i])
    for i in range(k[1]):
        G.add_node((1, i), demand=l[1][i])

    # excess
    for i in range(k[0]):
        G.add_edge(0, (0, i), weight=1, capacity=infinity)
    for i in range(k[1]):
        G.add_edge((1, i), 1, weight=1, capacity=infinity)

    # deficit
    for i in range(k[0]):
        G.add_edge((0, i), 0, weight=1, capacity=infinity)
    for i in range(k[1]):
        G.add_edge(1, (1, i), weight=1, capacity=infinity)

    # x_{i1,i2}
    l = np.array(np.concatenate(l))
    n_prime = extract_n_prime(L_prime, k)
    A = compute_A(L_prime, k, n_prime)
    U = compute_U(A, k)

    for i1 in range(k[0]):
        for i2 in range(k[1]):
            if U[i1, i2] > 0:
                G.add_edge((0, i1), (1, i2), capacity=U[i1, i2], weight=0)

    return G


def min_imbalance_networkx_extract_result(dc):
    s = 0
    for source, dests in dc.items():
        # we are interested in counting only d and e
        for dest, value in dests.items():
            if not isinstance(dest, tuple) or not isinstance(source, tuple):
                s += value
    return s


def min_imbalance_solver_networkx(l, L_prime, verbose=False):
    net = min_imbalance_network(l, L_prime)

    if verbose:
        for node in net.nodes(data=True):
            print("- o {}".format(node))
        for edg in net.edges(data=True):
            print("- > {}".format(edg))

    dc = nx.min_cost_flow(net)
    return min_imbalance_networkx_extract_result(dc)

### Minimum cost network flow (Google OR-Tools)

In [8]:
def convert_networkx_to_ortools(net, ortools_flow_obj):
    mapping = {}
    i = 0

    # weight and capacity
    for source, dest, data in net.edges(data=True):
        if source not in mapping:
            mapping[source] = i
            i += 1
        if dest not in mapping:
            mapping[dest] = i
            i += 1

        assert int(data["capacity"]) == data["capacity"]
        assert int(data["weight"]) == data["weight"]

        arc = ortools_flow_obj.AddArcWithCapacityAndUnitCost(
            mapping[source], mapping[dest], int(data["capacity"]), int(data["weight"])
        )

    for node, data in net.nodes(data=True):
        assert int(data["demand"]) == data["demand"]
        ortools_flow_obj.SetNodeSupply(mapping[node], -int(data["demand"]))

In [9]:
def min_imbalance_solver_google(l, L_prime):
    G = min_imbalance_network(l, L_prime)

    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    convert_networkx_to_ortools(G, min_cost_flow)

    status = min_cost_flow.Solve()
    if status == min_cost_flow.INFEASIBLE:
        print("Infeasible")
    elif status == min_cost_flow.UNBALANCED:
        print("Unbalanced")
    else:
        return min_cost_flow.OptimalCost()

### Minimum network flow with $q \neq n$ (NetworkX)

In [10]:
def compute_zero_cost_arc_capacity(li, q, n):
    return floor(q / n * li)


def compute_one_capacity_arc_weight(li, q, n):
    t = q / n * li
    return 1 - (t - floor(t))

In [11]:
def binary_search(ls, left, right, condition):
    if left == right:
        return left

    m = floor((left + right) / 2)
    v = condition(ls[m])
    if v == 0:
        return m
    if v < 0:
        return binary_search(ls, m + 1, right, condition)
    return binary_search(ls, left, m, condition)


def find_5(x):
    if x == 5:
        return 0
    if x < 5:
        return -1
    return 1


l = [1, 2, 3, 4, 5, 6]

l[binary_search(l, 0, 6, find_5)]

5

In [36]:
def condition(power, t):
    k = power * t
    if k / 10 == int(k / 10):
        return 1
    if k == int(k):
        return 0
    return -1


# find the smallest power of 10 which should be used to multiply in order to obtain
# integer weights for all the arcs
def minimum_scaling(ls, max_power=10):
    powers = np.power(10, np.arange(max_power + 1))

    top = 0
    for t in ls:
        if t == int(t):
            continue
        cnd = partial(condition, t=t)
        tp = binary_search(powers, 0, len(powers), cnd)
        if tp >= max_power:
            print("Ceiling reached with {}".format(t))
            tp = max_power - 1
        top = max(top, tp)
    return top


# we set all data explicitly because this will be used by another solver.
# max_power is the maximum exponent such that 10^max_power is an allowed multiplier to
# scale weights in the graph in order to have integer weights.
# this function returns the graph, and the multiplier used for the weights
def general_min_imbalance_network(l, L_prime, q, max_power):
    G = nx.DiGraph()

    G.add_node(0, demand=-q)
    G.add_node(1, demand=q)

    n = extract_n(l)
    k = extract_k(l)
    P = len(k)
    assert P == 2

    all_weights = [q / n] + [
        compute_one_capacity_arc_weight(l[p][i], q, n) * 2 / q
        for p in range(P)
        for i in range(k[p])
    ]
    powers = np.power(10, np.arange(max_power + 1))
    top = minimum_scaling(all_weights, max_power)

    for i in range(k[0]):
        G.add_node((0, i), demand=0)
    for i in range(k[1]):
        G.add_node((1, i), demand=0)

    # cost zero
    for i in range(k[0]):
        G.add_edge(
            0,
            (0, i),
            weight=1,
            capacity=compute_zero_cost_arc_capacity(l[0][i], q, n) * powers[top],
        )
    for i in range(k[1]):
        G.add_edge(
            (1, i),
            1,
            weight=1,
            capacity=compute_zero_cost_arc_capacity(l[1][i], q, n) * powers[top],
        )

    # capacity one
    for i in range(k[0]):
        G.add_edge(
            0,
            (0, i),
            weight=compute_one_capacity_arc_weight(l[0][i], q, n) * 2 / q * powers[top],
            capacity=1,
        )
    for i in range(k[1]):
        G.add_edge(
            (1, i),
            1,
            weight=compute_one_capacity_arc_weight(l[1][i], q, n) * 2 / q * powers[top],
            capacity=1,
        )

    # capacity infinity
    for i in range(k[0]):
        G.add_edge(0, (0, i), weight=2 / q * powers[top], capacity=infinity)
    for i in range(k[1]):
        G.add_edge((1, i), 1, weight=2 / q * powers[top], capacity=infinity)

    # x_{i1,i2}
    l = np.array(np.concatenate(l))
    n_prime = extract_n_prime(L_prime, k)
    A = compute_A(L_prime, k, n_prime)
    U = compute_U(A, k)

    for i1 in range(k[0]):
        for i2 in range(k[1]):
            if U[i1, i2] > 0:
                G.add_edge((0, i1), (1, i2), capacity=U[i1, i2], weight=0)

    return G, powers[top]


def general_min_imbalance_networkx_extract_result(G, dc):
    s = 0
    for source, dests in dc.items():
        # we are interested in counting only d and e
        for dest, value in dests.items():
            if not isinstance(dest, tuple) or not isinstance(source, tuple):
                s += value * G.get_edge_data(source, dest)["weight"]
    return s


def general_min_imbalance_solver_networkx(l, L_prime, q, verbose=False, max_power=10):
    net, scale = general_min_imbalance_network(l, L_prime, q, max_power=max_power)

    if verbose:
        for node in net.nodes(data=True):
            print("- o {}".format(node))
        for edg in net.edges(data=True):
            print("- > {}".format(edg))

    dc = nx.min_cost_flow(net)
    return general_min_imbalance_networkx_extract_result(net, dc) / scale

### Minimum network flow with $q \neq n$ (Google OR-Tools)

In [30]:
def general_min_imbalance_solver_google(l, L_prime, q, max_power=10):
    G, scale = general_min_imbalance_network(l, L_prime, q, max_power=max_power)

    min_cost_flow = pywrapgraph.SimpleMinCostFlow()
    convert_networkx_to_ortools(G, min_cost_flow)

    status = min_cost_flow.Solve()
    if status == min_cost_flow.INFEASIBLE:
        print("Infeasible")
    elif status == min_cost_flow.UNBALANCED:
        print("Unbalanced")
    else:
        return min_cost_flow.OptimalCost() / (scale)

## Experimental results

### Random dataset generator

In [14]:
def group_by(a):
    a = a[a[:, 0].argsort()]
    return np.split(a[:, 1], np.unique(a[:, 0], return_index=True)[1][1:])


def random_Lprime(P, n_prime, k):
    L_prime = [None for p in range(P)]
    for p in range(P):
        # randomly select a level for each control value
        choice = np.random.choice(np.arange(k[p]), size=n_prime)
        indexed_choices = np.hstack([choice[:, None], np.arange(n_prime)[:, None]])
        gb = group_by(indexed_choices)

        un = np.unique(choice)
        i = 0
        for u in un:
            for j in range(i, u):
                gb.insert(i, [])
            i = u + 1
        for j in range(i, k[p]):
            gb.append([])

        L_prime[p] = gb
    return L_prime


def print_Lprime(L_prime):
    for p in range(len(L_prime)):
        print(p, L_prime[p])

In [15]:
# ----- VARIABLES -------
n_prime = 5

# n is equal to the size of the treatmeant sample
l = (np.array([5, 3, 5, 0, 0, 0, 0, 0]), np.array([4, 4, 5, 0, 0]))
for li in l:
    assert sum(li) == sum(l[0])
# -----------------------

P = len(l)
L_prime = random_Lprime(P, n_prime, extract_k(l))

### Non-random dataset

In [37]:
l = [[1, 0, 0, 2, 1], [3, 0, 1]]
L_prime = [
    [[0, 8, 10], [1, 2, 9], [3, 4], [5, 11], [6, 7]],
    [[1, 3, 4, 6], [5, 9, 10, 8], [0, 2, 7, 11]],
]

print(min_imbalance_solver(l, L_prime))
print(min_imbalance_solver_alt(l, L_prime))
print(min_imbalance_solver_mcnf(l, L_prime))
print(min_imbalance_solver_networkx(l, L_prime))
print(min_imbalance_solver_google(l, L_prime))
print(general_min_imbalance_solver_networkx(l, L_prime, 4))
print(general_min_imbalance_solver_google(l, L_prime, 4))

(array([ 1.,  0., -0.,  0., -0.,  1.,  1., -0., -0., -0.,  0.,  1.]), 4.0)
(array([ 1.,  0., -0.,  0., -0.,  1.,  1., -0., -0., -0.,  0.,  1.]), 4.0)
(array([1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.]), 4.0)
4
4
4.0
4.0
